# EVI Analysis

* Fetch the GLC-FCS30D using Google Earth Engine
* Combine all the tiles in GLC into a global tile
* Given that each tile has multiple bands that represent multiple year, modify the data into an Image Collection
* Select all the cropland values only (10, 11, 12, and 20)
* Create mask for every cropland value and the overall cropland mask
* Fetch the EVI data and Bolivia boundaries
* Mask the EVI data using the cropland mask
* Visualize the EVI based on each cropland per year and clip to Bolivia only
* Export the map visualization as as static map
* Create a zonal stats for each region in Bolivia by aggregating into mean

## Import library and data

In [1]:
from pathlib import Path
import json
import logging

import ee
import geemap
import geemap.colormaps as cm
import geopandas as gpd
import requests

logger = logging.getLogger(__name__)


PROJECT_ID = "ferrous-weaver-462011-d2"
ee.Initialize(project=PROJECT_ID)

PROJECT_ROOT = Path().cwd().parent
DATA_PATH = PROJECT_ROOT / "data"
BOUNDARIES_PATH = DATA_PATH / "boundaries"

ee.Initialize()

/Users/farhanreynaldo/Documents/world-bank/git-repo/bolivia-economic-monitor/.venv/lib/python3.11/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## Functions

In [2]:
def load_geojson_to_ee(
    geojson_path: str | Path,
) -> ee.FeatureCollection:
    """Load a GeoJSON file to Earth Engine.

    Args:
        geojson_path: The path to the GeoJSON file.

    Returns:
        An ee.FeatureCollection equivalent to the GeoJSON file.
    """
    if isinstance(geojson_path, str):
        geojson_path = Path(geojson_path)

    if not geojson_path.exists():
        raise FileNotFoundError(f"File not found: {geojson_path}")

    with open(geojson_path) as f:
        geojson = json.load(f)

    fc = ee.FeatureCollection(geojson)

    return fc


def fetch_boundaries(
    iso3_code: str,
    adm_level: int,
    release_type: str = "gbOpen",
    output_dir: str | Path = "data/boundaries",
):
    cache_dir = Path(output_dir)
    cache_dir.mkdir(parents=True, exist_ok=True)
    cache_file = cache_dir / f"{iso3_code}_ADM{adm_level}_{release_type}.geojson"

    if cache_file.exists():
        logger.info(f"Loading boundaries from cache: {cache_file}")
        return load_geojson_to_ee(cache_file)

    url = f"https://www.geoboundaries.org/api/current/{release_type}/{iso3_code}/ADM{adm_level}"
    logger.info(f"Fetching boundaries from API: {url}")
    response = requests.get(url)
    response.raise_for_status()
    download_url = response.json()["gjDownloadURL"]
    logger.info(f"Downloading GeoJSON from: {download_url}")
    gdf = gpd.read_file(download_url)
    gdf.to_file(cache_file, driver="GeoJSON")
    logger.info(f"Boundaries saved to cache: {cache_file}")
    return load_geojson_to_ee(cache_file)


def bitwiseExtract(value: ee.Image, fromBit: int, toBit: int | None = None) -> ee.Image:
    """Extract bits from a binary image."""
    toBit = fromBit if toBit is None else toBit
    maskSize = ee.Number(1).add(toBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return value.rightShift(fromBit).bitwiseAnd(mask)


def apply_modisQA_mask(image: ee.Image):
    sqa = image.select("SummaryQA")
    dqa = image.select("DetailedQA")
    viQualityFlagsS = bitwiseExtract(sqa, 0, 1)
    viQualityFlagsD = bitwiseExtract(dqa, 0, 1)
    viSnowIceFlagsD = bitwiseExtract(dqa, 14)
    # Good data, use with confidence
    mask = (
        viQualityFlagsS.eq(0)
        .And(viQualityFlagsD.eq(0))
        .And(viQualityFlagsS.eq(1))
        .And(viQualityFlagsD.eq(1))
        .And(viSnowIceFlagsD)
        .eq(0)
    )
    return image.updateMask(mask)


def load_evi_data(
    start_date: str | ee.Date,
    end_date: str | ee.Date,
    apply_crop_mask: bool = True,
) -> ee.ImageCollection:
    """
    Load EVI data from MODIS.

    Args:
        start_date: Start date.
        end_date: End date.
        apply_crop_mask: Whether to apply crop mask.

    Returns:
        EVI image collection.
    """

    terra = (
        ee.ImageCollection("MODIS/061/MOD13Q1")
        .select(["EVI", "SummaryQA", "DetailedQA"])
        .filterDate(start_date, end_date)
    )

    aqua = (
        ee.ImageCollection("MODIS/061/MYD13Q1")
        .select(["EVI", "SummaryQA", "DetailedQA"])
        .filterDate(start_date, end_date)
    )

    mod13q1_QC = terra.map(apply_modisQA_mask)
    myd13q1_QC = aqua.map(apply_modisQA_mask)
    mxd13q1_cleaned = mod13q1_QC.select("EVI").merge(myd13q1_QC.select("EVI"))
    mxd13q1 = mxd13q1_cleaned.sort("system:time_start")
    return mxd13q1

In [ ]:
bolivia_geom = fetch_boundaries("BOL", 0, "gbOpen")
evi_data = load_evi_data("2014-01-01", "2022-12-31", apply_crop_mask=False)
center = [-16.2902, -63.5887]

bolivia_bbox = bolivia_geom.geometry().bounds()
bolivia_bbox = ee.Geometry.BBox(-22.8982742, -9.6689438, -69.6450073, -57.453)

## load and preprocessing the GLC

In [5]:
annual = ee.ImageCollection("projects/sat-io/open-datasets/GLC-FCS30D/annual")

class_values = [
    10,
    11,
    12,
    20,
    51,
    52,
    61,
    62,
    71,
    72,
    81,
    82,
    91,
    92,
    120,
    121,
    122,
    130,
    140,
    150,
    152,
    153,
    181,
    182,
    183,
    184,
    185,
    186,
    187,
    190,
    200,
    201,
    202,
    210,
    220,
    0,
]
class_names = [
    "Rainfed_cropland",
    "Herbaceous_cover_cropland",
    "Tree_or_shrub_cover_cropland",
    "Irrigated_cropland",
    "Open_evergreen_broadleaved_forest",
    "Closed_evergreen_broadleaved_forest",
    "Open_deciduous_broadleaved_forest",
    "Closed_deciduous_broadleaved_forest",
    "Open_evergreen_needle_leaved_forest",
    "Closed_evergreen_needle_leaved_forest",
    "Open_deciduous_needle_leaved_forest",
    "Closed_deciduous_needle_leaved_forest",
    "Open_mixed_leaf_forest",
    "Closed_mixed_leaf_forest",
    "Shrubland",
    "Evergreen_shrubland",
    "Deciduous_shrubland",
    "Grassland",
    "Lichens_and_mosses",
    "Sparse_vegetation",
    "Sparse_shrubland",
    "Sparse_herbaceous",
    "Swamp",
    "Marsh",
    "Flooded_flat",
    "Saline",
    "Mangrove",
    "Salt_marsh",
    "Tidal_flat",
    "Impervious_surfaces",
    "Bare_areas",
    "Consolidated_bare_areas",
    "Unconsolidated_bare_areas",
    "Water_body",
    "Permanent_ice_and_snow",
    "Filled_value",
]
class_colors = [
    "#ffff64",
    "#ffff64",
    "#ffff00",
    "#aaf0f0",
    "#4c7300",
    "#006400",
    "#a8c800",
    "#00a000",
    "#005000",
    "#003c00",
    "#286400",
    "#285000",
    "#a0b432",
    "#788200",
    "#966400",
    "#964b00",
    "#966400",
    "#ffb432",
    "#ffdcd2",
    "#ffebaf",
    "#ffd278",
    "#ffebaf",
    "#00a884",
    "#73ffdf",
    "#9ebb3b",
    "#828282",
    "#f57ab6",
    "#66cdab",
    "#444f89",
    "#c31400",
    "#fff5d7",
    "#dcdcdc",
    "#fff5d7",
    "#0046c8",
    "#ffffff",
    "#ffffff",
]

# Mosaic the data into a single image
annual_mosaic = annual.mosaic()

years_list = [str(y) for y in range(2000, 2023)]
annual_mosaic_renamed = annual_mosaic.rename(years_list)

# Convert the multiband image to an ImageCollection
images = []
for year_str in years_list:
    year_int = int(year_str)
    date = ee.Date.fromYMD(year_int, 1, 1)
    image = annual_mosaic_renamed.select([year_str]).set(
        {"system:time_start": date.millis(), "system:index": year_str, "year": year_int}
    )
    images.append(image)

mosaics_col = ee.ImageCollection.fromImages(images)

recode = False
if recode:
    new_class_values = list(range(1, len(class_values) + 1))

    def remap_classes(image):
        return image.remap(class_values, new_class_values).rename("classification")

    mosaics_col = mosaics_col.map(remap_classes)

In [15]:
## Parameters

YEAR = 2014
CROPLAND_CLASS = 11

cropland_values = [10, 11, 12, 20]  # Values for cropland classes

## Single cropland

In [14]:
image_sample = mosaics_col.filter(ee.Filter.eq("year", YEAR)).mosaic()
crop_sample = image_sample.updateMask(image_sample.eq(CROPLAND_CLASS))

m = geemap.Map(center=center, zoom=6)
m.addLayer(
    crop_sample,
    {},
    f"Land Cover {YEAR} - Cropland (Class {CROPLAND_CLASS})",
)
m

Map(center=[-16.2902, -63.5887], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

## Overall cropland

In [16]:
image_sample = mosaics_col.filter(ee.Filter.eq("year", YEAR)).mosaic()

crop_mask = ee.Image(0)  # Start with an empty mask (all 0s)
for val in cropland_values:
    crop_mask = crop_mask.Or(image_sample.eq(val))

crop_sample = image_sample.updateMask(crop_mask)

m = geemap.Map(center=center, zoom=6)
m.addLayer(
    crop_sample,
    {},
    f"Land Cover {YEAR} - Cropland (Classes {', '.join(map(str, cropland_values))})",
    True,
)
m

Map(center=[-16.2902, -63.5887], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

## EVI based on single cropland

In [18]:
vis_params = {"min": -2000, "max": 10000, "palette": cm.get_palette("Greens", 20)}

image_subset = mosaics_col.filter(ee.Filter.eq("year", YEAR)).mosaic()
crop_mask = image_subset.eq(CROPLAND_CLASS)
evi_subset = evi_data.filterDate(
    ee.Date.fromYMD(YEAR, 1, 1), ee.Date.fromYMD(YEAR, 12, 31)
).mean()
evi_subset = evi_subset.updateMask(crop_mask)

m = geemap.Map(center=center, zoom=6, basemap="CartoDB.Positron")
m.addLayer(
    evi_subset,
    vis_params,
    f"Land Cover {YEAR}",
)
m

Map(center=[-16.2902, -63.5887], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

## EVI based on overall cropland

In [19]:
vis_params = {"min": -2000, "max": 10000, "palette": cm.get_palette("Greens", 10)}

image_subset = mosaics_col.filter(ee.Filter.eq("year", YEAR)).mosaic()

crop_mask = ee.Image(0)
for val in cropland_values:
    crop_mask = crop_mask.Or(image_subset.eq(val))

evi_subset = evi_data.filterDate(
    ee.Date.fromYMD(YEAR, 1, 1), ee.Date.fromYMD(YEAR, 12, 31)
).mean()

evi_subset = evi_subset.updateMask(crop_mask)

m = geemap.Map(center=center, zoom=6, basemap="CartoDB.Positron")
m.addLayer(
    evi_subset,
    vis_params,
    f"Land Cover {YEAR}",
)
m

Map(center=[-16.2902, -63.5887], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…